In [ ]:
import matplotlib.pyplot as plt
import numpy
from sklearn.model_selection import train_test_split
import pandas
from imblearn.over_sampling import SMOTE, ADASYN, SVMSMOTE, BorderlineSMOTE
import sklearn
import seaborn
import argparse
import pickle
import sklearn
from keras.utils import to_categorical
from sklearn.ensemble import RandomForestClassifier
from keras.utils import to_categorical

# Argument Parsing

In [ ]:
def pArgs(args):
    output = argparse.ArgumentParser('Run predictor for specific data on cll data.')

    output.add_argument('data', type=str, help='Path to data file.')
    output.add_argument('labels', type=str, help='Path to label file.')
    output.add_argument('--full_data', type=str, help='Path to full dataset to make predictions.', default=None)
    output.add_argument('--save-path', type=str, help='Path to save plots.', default=None)
    output.add_argument('--crossvalidate', type=int, help='Number of crossvalidation folds for metrics. 0 for no crossvalidation.', default=5)
    output.add_argument('--verbose', action='store_true', help='Show status of analysis.')

    return output.parse_args(args[1:])

# Core crossvalidation function
This function loads data and iterates over the oversamples and does the splitted training

In [ ]:
def load(path_x, path_y, split=0.15, seed=0):
    x = pandas.read_csv(path_x, index_col=0)
    y = pandas.read_csv(path_y, index_col=0)

    if split == 0:
        return x, y
    else:
        return train_test_split(x, y, test_size=0.15, random_state=seed)
    
def run_cross(args, target_func, name, split=0.15, seed=0, scaler_class=sklearn.preprocessing.StandardScaler, save_path=None, **kwargs_func):
    data_original, labels_original = load(args.data, args.labels, split=0)

    res = list()

    for overs in ["", "SMOTE", "ADASYN", "SVMSMOTE", "BorderlineSMOTE"]:
        print('Oversampling: ', overs)
        try:
            temp = train_cv(args, data_original, labels_original, target_func, overs, n_splits=args.crossvalidate)
            res.append( (overs + ' Raw ' + name, *temp) )
            if args.verbose:
                print('raw done')
        except:
            raise

        try:
            temp = train_cv(args, data_original, labels_original, target_func, overs, n_splits=args.crossvalidate, scaler=scaler_class())
            res.append( (overs + ' Scaled ' + name, *temp) )
            if args.verbose:
                print('raw done')
        except:
            raise

    try:
        best = sorted(res, key=lambda x: x[1][0], reverse=True)[0]
        draw(path=args.save_path + '/plots/' + name + '_' + str(best[2][0]) if args.save_path is not None else None, arguments=args, *res)
        print('Best:', best[0], best[1][0])
    except:
        print('No data')
        raise

# Plotting
Given the output of a crossvalidating training set plot the resulting confusion matrix and others.

In [ ]:
def draw(*args, path=None, arguments=None):
    n = int(numpy.ceil(numpy.sqrt(len(args))))
    f, ax = plt.subplots(int(numpy.ceil(len(args)/n)), n)
    f.set_size_inches(16,9)
    f.set_dpi(500)
    assert(len(args) <= n*numpy.ceil(len(args)/n))

    ax = ax.flatten()
    
    for i in range(len(args)):
        confus1 = args[i][1] if arguments is None or arguments.crossvalidate == 0 else args[i][1][-2] 
        confus = args[i][1] if arguments is None or arguments.crossvalidate == 0 else args[i][1][-2] 
        confus = numpy.nan_to_num(confus / confus.sum(axis=0))
        seaborn.heatmap(confus, vmax=1.0, xticklabels=range(1, confus.shape[0]+1), yticklabels=range(1, confus.shape[0]+1), cmap='YlOrRd', ax=ax[i], annot=confus1, fmt='d', square=True)
        if arguments is None or arguments.crossvalidate == 0:
            ax[i].set_title(args[i][0] + '\nAccuracy: ' + str(args[i][2]) + ' - Recall:' + str(args[i][3]) + ' - F1: ' + str(args[i][4]))
        else:
            ax[i].set_title(args[i][0] + '\nAccuracy: ' + str(args[i][1][0]) + ' - Recall:' + str(args[i][1][1]) + ' - F1: ' + str(args[i][1][2]) + '\nAccuracy: ' + str(args[i][2][0]) + '+/-' + str(args[i][2][1]) + '\nRecall: ' + str(args[i][3][0]) + '+/-' + str(args[i][3][1]) + '\nF1: ' + str(args[i][3][0]) + '+/-' + str(args[i][3][1]))
            
        ax[i].set_ylabel('Real')
        ax[i].set_xlabel('Predicted')

    f.set_tight_layout(True)
    plt.tight_layout()
    if path is not None:
        plt.savefig(path + '_plot.pdf')
    else:
        plt.show()
        
    plt.close('all')

# Train function
This functions manages the crossvalidation splits for a given set.

In [ ]:
def train_cv(args, X, Y, target_f, oversampler, n_splits=10, scaler=None):
    cv = sklearn.model_selection.StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    res = list()
    
    for i, (train1, test1) in enumerate(cv.split(X, Y)):
        x_train = X.iloc[train1]
        y_train = Y.iloc[train1]
        x_test = X.iloc[test1]
        y_test = Y.iloc[test1]
        
        if scaler is not None:
            if x_train.columns.str.contains('vsAll').any():
                c = x_train.columns.str.contains('vsAll')
                x_train[x_train.columns[c]] = x_train[x_train.columns[c]].div(x_train[x_train.columns[c]].sum(axis=1), axis=0) * 1000000
                x_test[x_test.columns[c]] = x_test[x_test.columns[c]].div(x_test[x_test.columns[c]].sum(axis=1), axis=0) * 1000000
                
                x_train[x_train.columns[~c]] = x_train[x_train.columns[~c]].div(x_train[x_train.columns[~c]].sum(axis=1), axis=0) * 1000000
                x_test[x_test.columns[~c]] = x_test[x_test.columns[~c]].div(x_test[x_test.columns[~c]].sum(axis=1), axis=0) * 1000000
                
            else:
                x_train = x_train.div(x_train.sum(axis=1), axis=0) * 1000000
                x_test = x_test.div(x_test.sum(axis=1), axis=0) * 1000000
            
        pred, model = train(x_train, y_train, x_test, y_test, target_f, oversampler=oversampler)

        full = pandas.read_csv(args.full_data, index_col=0).T
        full = full[x_train.columns]
        
        if scaler is not None:
            if full.columns.str.contains('vsAll').any():
                c = full.columns.str.contains('vsAll')
                full[full.columns[c]] = full[full.columns[c]].div(full[full.columns[c]].sum(axis=1), axis=0) * 1000000
                
                full[full.columns[~c]] = full[full.columns[~c]].div(full[full.columns[~c]].sum(axis=1), axis=0) * 1000000
                
            else:
                full = full.div(full.sum(axis=1), axis=0) * 1000000
        
        res2 = pandas.DataFrame(model.predict_proba(full))
        res2.index = full.index
        res2.columns = ['Cluster {}'.format(i) for i in range(len(res2.columns))]
        res2['Pred_Cluster'] = model.predict(full)
        
        
        if scaler is not None and False:
            model = sklearn.pipeline.Pipeline([('scaler', scaler), ('model', model)])
        
        confus = sklearn.metrics.confusion_matrix(y_test, pred)
        acc = int(sklearn.metrics.accuracy_score(y_test, pred) * 1000) / 1000
        recall = int(sklearn.metrics.recall_score(y_test, pred, average='micro') * 1000) / 1000
        f1 = int(sklearn.metrics.f1_score(y_test, pred, average='micro' ) * 1000) / 1000
        
        res.append( (acc, recall, f1, confus, model) )
        name = target_f.__name__ + '_' + oversampler + '_split_' + str(i)
        if scaler is not None:
            name = name + '_scaled'
        if oversampler != '' and oversampler is not None:
            name = name + '_' + oversampler
            
        res2.to_csv(args.save_path + '/' + name + '_full_predict_fold_' + str(i) + '.csv')
            
        save_soft_clustering_predictions_split(args, model, name, X, Y, train1, test1, acc) 
        with open(args.save_path + '/models/' + name + '_' + str(acc) + '.pickle', 'wb') as f:
            pickle.dump(model, f)
        
    plt.clf()
    plt.plot(range(len(res)), [x[0] for x in res])
    plt.xlabel('Fold #')
    plt.ylabel('Accuracy')
    plt.title('Fold accuracy for {}'.format(target_f.__name__))
    name = args.save_path + '/plots/fold_accs_' + target_f.__name__
    if oversampler is not None:
        name = name + '_' + oversampler
        
    if scaler is not None:
        name = name + '_scaled'
    plt.savefig(name) 
    
    res = sorted(res, key=lambda x: x[0], reverse=True)
    best = res[0]
    
    acc_m = numpy.format_float_positional(numpy.mean([x[0] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    acc_s = numpy.format_float_positional(numpy.std([x[0] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    
    rec_m = numpy.format_float_positional(numpy.mean([x[1] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    rec_s = numpy.format_float_positional(numpy.std([x[1] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    
    f1_m = numpy.format_float_positional(numpy.mean([x[2] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    f1_s = numpy.format_float_positional(numpy.std([x[2] for x in res]), precision=2, unique=False, fractional=False, trim='k')
    
    return best, (acc_m, acc_s), (rec_m, rec_s), (f1_m, f1_s)

# Training
Deals with routing the data to the actual training function and returns results and trained model.

In [ ]:
def train(x, y, xt, yt, target_function, oversampler=None, pandas_column='bnmf_expression_cluster', **f_kwargs):
    oversamplers = {"SMOTE":SMOTE, "ADASYN":ADASYN, "SVMSMOTE":SVMSMOTE, "BorderlineSMOTE":BorderlineSMOTE}
    x1, y1 = x, y
    try:
        if oversampler is not None and oversampler != "":
            if pandas_column != None:
                x1, y1 = oversamplers[oversampler](random_state=0).fit_resample(x1, y1[pandas_column])
            else:
                x1, y1 = oversamplers[oversampler](random_state=0).fit_resample(x1, y1)
                
            x1 = x1.sample(frac=1)
            y1 = y1.iloc[x1.index]
    except:
        print('no samples generated extra')
        
    return target_function(x1, y1, xt, yt, **f_kwargs)

# Final Plots
For a full dataset and a trained model calculate all scores.

In [ ]:
def save_soft_clustering_predictions_split(args, model, name, data, labels, split_train, split_test, acc):
    def f(x):
        return -x.sort_values(inplace=False, ascending=False)[:2].diff()[1]
        
    df = model.predict_proba(data)
    df = pandas.DataFrame(df)
    df.columns = ['cluster_' + str(i) for i in model.classes_]
    df['diff_top2'] = df.apply(f, axis=1)
    
    df['pred'] = model.predict(data)
    
    df.index = data.index
    df.index.name = 'sample_id'
    df['real_cluster'] = labels
    is_test = numpy.zeros(df.shape[0])
    is_test[split_test] = 1
    
    df['test_sample'] = is_test
    
    df.to_csv(args.save_path + '/soft_clust/' + name + '_' + str(acc) + '.csv')
    
    plt.clf()
    df['yis'] = (df['real_cluster'] == df['pred']).astype('int')
    df.groupby('yis').boxplot(column=['diff_top2'])
    plt.savefig(args.save_path + '/plots/box_diff_' + name + '_' + str(acc) + '.png')

# Models
This actually instances the models and deals with the __fit()__ function.

In [ ]:
def rf2(x, y, xt, yt, estimators=1000):
    y1 = to_categorical(y)
    yt1 = to_categorical(yt)

    rf = RandomForestClassifier(n_estimators = estimators, random_state = 236, class_weight='balanced_subsample')

    rf.fit(x, y.to_numpy().ravel())

    y_pred = rf.predict(xt)

    return y_pred, rf

def rf(x, y, xt, yt, estimators=500):
    y1 = to_categorical(y)
    yt1 = to_categorical(yt)

    rf = RandomForestClassifier(n_estimators = estimators, random_state = 236, class_weight='balanced_subsample')

    rf.fit(x, y.to_numpy())    
    
    y_pred = rf.predict(xt)

    return y_pred, rf


# Preparing dataset
This selects samples and genes + genesets for the pipeline.  
This is not a part of the notebook but added for storage.

In [ ]:
#Input: Full TPM/counts matrix
#Output: Subsets to genes and samples of interest (used in this notebook)
def parse_dataset(data_path, geneset_list_path, samples_path, labels_path, genes_path):
    data = pandas.read_csv(data_path, index_col=0)
    genes = pandas.read_csv(geneset_list_path, sep='\t') if geneset_list_path != 'NONE' else None
    samples = pandas.read_csv(samples_path, header=None)
    labels = pandas.read_csv(labels_path, index_col=0, sep='\t')
    rest_genes = pandas.read_csv(genes_path, header=None)
    
    output = data.T[rest_genes[0].values]
    #data = data / data.sum()
    
    if genes is not None:
        d = dict()
        
        for i, j in genes.groupby('geneset'):
            d[i] = data.loc[j['gene']].sum()
            
        output = pandas.concat([pandas.DataFrame().from_dict(d), output], axis=1)
        
    output = output.loc[samples[0].values]
    
    output.to_csv(sys.argv[6] + '.csv')
    labels.loc[samples[0].values].to_csv(sys.argv[6] + '_labels.csv')

# RUN PARAMS

In [ ]:
PATH_DATA = ""
PATH_LABELS = ""
PATH_FULL_DATA = ""
PATH_SAVE = "/tmp"

!mkdir -p {PATH_SAVE}/models
!mkdir -p {PATH_SAVE}/plots
!mkdir -p {PATH_SAVE}/soft_clust

command_line_str = "program.py " + PATH_DATA + " " + PATH_LABELS + " --full_data " + PATH_FULL_DATA + " --save-path " + PATH_SAVE

args = pArgs(command_line_str.split(' '))

run_cross(args, rf, 'RandomForestSmall')
run_cross(args, rf2, 'RandomForestBig')